# Agentic RAG with LangChain: Complete Guide

## What is Agentic RAG?

**Agentic RAG** combines Retrieval-Augmented Generation (RAG) with autonomous agents that can:
- **Decide** which tools to use based on the query
- **Reason** about the best approach to answer questions
- **Combine** multiple information sources (vector databases, web search, APIs)
- **Plan** multi-step operations to solve complex queries

### Traditional RAG vs Agentic RAG

| Traditional RAG | Agentic RAG |
|----------------|-------------|
| Single retrieval path | Multiple tool options |
| No reasoning about source | Intelligently selects best source |
| Static workflow | Dynamic decision-making |
| Limited to vector DB | Can use web search, APIs, calculators, etc. |

---

## Setup and Dependencies

Install required packages for this demonstration:
```bash
pip install langchain langchain-openai langchain-community langgraph chromadb tavily-python
```

**Note:** This notebook uses the **latest LangChain v1 API** with `create_agent`, which is built on top of LangGraph for better reliability and simpler interface.

In [1]:
# Import necessary libraries
import os
from dotenv import load_dotenv

# LangChain core imports (v1 API)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain.tools import tool

# NEW: LangChain v1 agent API
from langchain.agents import create_agent

# Load environment variables
load_dotenv()

# Verify API keys are loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment variables")
if not os.getenv("TAVILY_API_KEY"):
    raise ValueError("TAVILY_API_KEY not found in environment variables")

print("✅ All dependencies imported successfully!")
print("📦 Using LangChain v1 API with create_agent")

✅ All dependencies imported successfully!
📦 Using LangChain v1 API with create_agent


## Step 1: Create Sample Documents and Vector Database

We'll create a knowledge base about **AI and Machine Learning concepts** using ChromaDB.

### Why ChromaDB?
- Lightweight and easy to use
- Persistent storage option
- Fast similarity search
- Perfect for local development

In [2]:
# Create sample documents for our knowledge base
documents = [
    Document(
        page_content="""LangChain is a framework for developing applications powered by language models. 
        It enables applications that are context-aware and can reason. LangChain provides standard, 
        extendable interfaces and integrations for various components like model I/O, retrieval, 
        agents, and memory. The framework was created by Harrison Chase and launched in October 2022.""",
        metadata={"source": "langchain_docs", "topic": "framework"}
    ),
    Document(
        page_content="""Retrieval-Augmented Generation (RAG) is a technique that combines the power of 
        large language models with external knowledge retrieval. It works by first retrieving relevant 
        documents from a knowledge base, then using those documents as context for the LLM to generate 
        an answer. This approach reduces hallucinations and provides more accurate, grounded responses.""",
        metadata={"source": "ai_concepts", "topic": "rag"}
    ),
    Document(
        page_content="""Vector databases store data as high-dimensional vectors, which are mathematical 
        representations of data features. They enable semantic search by finding similar vectors using 
        distance metrics like cosine similarity. Popular vector databases include Chroma, Pinecone, 
        Weaviate, and Qdrant. These databases are essential for building RAG applications.""",
        metadata={"source": "ai_concepts", "topic": "vector_db"}
    ),
    Document(
        page_content="""LangChain agents are systems that use a language model as a reasoning engine to 
        determine which actions to take and in what order. Agents have access to tools (like search engines, 
        calculators, or databases) and can dynamically decide which tool to use based on the user's input. 
        The agent follows a reasoning loop: Thought -> Action -> Observation -> repeat until done.""",
        metadata={"source": "langchain_docs", "topic": "agents"}
    ),
    Document(
        page_content="""GPT-4 is OpenAI's fourth-generation language model, released in March 2023. 
        It's a multimodal model that can accept both image and text inputs and produce text outputs. 
        GPT-4 exhibits human-level performance on various professional and academic benchmarks. 
        It has improved reasoning capabilities and reduced hallucinations compared to GPT-3.5.""",
        metadata={"source": "ai_models", "topic": "gpt4"}
    )
]

print(f"📚 Created {len(documents)} sample documents")

📚 Created 5 sample documents


In [3]:
# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"📄 Split into {len(splits)} chunks")

📄 Split into 5 chunks


In [4]:
# Create embeddings and vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create Chroma vector database
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="agentic_rag_demo",
    persist_directory="./chroma_agentic_rag"
)

print("✅ Vector database created successfully!")
print(f"📊 Collection contains {vectorstore._collection.count()} vectors")

✅ Vector database created successfully!
📊 Collection contains 25 vectors


## Step 2: Create Tools for the Agent

### Tool 1: Retriever Tool
Searches through our local knowledge base (vector database) for relevant information.

### Tool 2: Tavily Web Search
Searches the web for current, up-to-date information when the local knowledge base doesn't have the answer.

### LangChain v1 Tool Pattern:
In the new API, we use the `@tool` decorator to create tools with clear names and descriptions that help the agent decide when to use them.

In [5]:
# Create retriever from vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Retrieve top 3 most similar documents
)

# Create retriever tool using @tool decorator (NEW in LangChain v1)
@tool
def knowledge_base_search(query: str) -> str:
    """Search through the local knowledge base containing information about LangChain, 
    RAG (Retrieval-Augmented Generation), vector databases, AI agents, and GPT models. 
    Use this tool when the question is about fundamental AI/ML concepts, LangChain framework, 
    or technical explanations that don't require real-time information."""
    
    docs = retriever.invoke(query)
    # Format the retrieved documents
    return "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

print("✅ Retriever tool created using @tool decorator")

✅ Retriever tool created using @tool decorator


In [6]:
# Create Tavily web search tool
web_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=False,
    include_images=False
)

print("✅ Web search tool created")

✅ Web search tool created


C:\Users\Vineet\AppData\Local\Temp\ipykernel_8564\2167215660.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


In [7]:
#create a calculator tool for addition
@tool
def add(a: int, b: int) -> int:
    """Add two numbers together. Use this when you need to calculate the sum of two numbers.
    Example: add(5, 3) returns 8"""
    return a + b


In [8]:
# Create a calculator tool for subtraction

@tool
def subtract(a: int, b: int) -> int:
    """Subtract b from a. Use this when you need to find the difference.
    Example: subtract(10, 3) returns 7"""
    return a - b

In [9]:
# Create a calculator tool for multiplication
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers. Use this when you need to find the product.
    Example: multiply(4, 5) returns 20"""
    return a * b

In [12]:
# Combine tools into a list
tools = [knowledge_base_search, web_search_tool,add,subtract,multiply]

print("\n🛠️  Available Tools for Agent:")
for i, tool in enumerate(tools, 1):
    print(f"  {i}. {tool.name}: {tool.description[:100]}...")


🛠️  Available Tools for Agent:
  1. knowledge_base_search: Search through the local knowledge base containing information about LangChain, 
    RAG (Retrieval-...
  2. tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need...
  3. add: Add two numbers together. Use this when you need to calculate the sum of two numbers.
    Example: a...
  4. subtract: Subtract b from a. Use this when you need to find the difference.
    Example: subtract(10, 3) retur...
  5. multiply: Multiply two numbers. Use this when you need to find the product.
    Example: multiply(4, 5) return...


## Step 3: Initialize the Language Model

We're using **GPT-4o** (GPT-4 Optimized) because:
- Superior reasoning capabilities
- Better function calling accuracy
- Improved multi-step planning
- Lower latency than GPT-4

In [13]:
# Initialize GPT-4o
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,  # Set to 0 for consistent, deterministic outputs
    streaming=True   # Enable streaming for better UX
)

print("✅ Language model initialized: GPT-4o")

✅ Language model initialized: GPT-4o


In [14]:
llm_with_tools = llm.bind_tools(tools)

## Step 4: Create Agent with System Prompt

### NEW LangChain v1 API - Simpler and More Powerful!

Instead of creating separate prompt templates and agent executors, we now use:
- **`create_agent()`** - Single function to create the agent
- **`system_prompt`** - Direct string for instructions (no template needed)
- Built on **LangGraph** - Automatic persistence, streaming, and human-in-the-loop

The agent will:
1. **Think** about which tool to use
2. **Execute** the appropriate tool
3. **Observe** the results
4. **Iterate** if needed
5. **Synthesize** a final answer

In [26]:
# Define system prompt for the agent with CURRENT DATE
from datetime import datetime

# Get current date to provide temporal context to the agent
current_date = datetime.now().strftime("%B %d, %Y")

system_prompt = f"""You are a helpful AI assistant with access to multiple tools. 
Your goal is to answer user questions by reasoning about which tool to use.

IMPORTANT: Today's date is {current_date}. Use this when searching for current information.

REASONING PROCESS:
1. Analyze the user's question carefully
2. Determine if you need information from the knowledge base or the web
3. Explain your reasoning before using a tool
4. Use tools to gather information
5. Synthesize the information into a clear, comprehensive answer

You can:
- Search a knowledge base about AI/ML concepts
- Search the web for current information  
- Perform mathematical calculations
- Query structured databases

Use your judgment to select appropriate tools based on the user's needs.

Always explain your thought process clearly."""

print(f"✅ System prompt defined with current date: {current_date}")

✅ System prompt defined with current date: December 16, 2025


### 🔑 Important: Providing Current Date Context

**Problem**: LLMs are trained on historical data and don't inherently know "today's date"

**Solution**: Inject the current date into the system prompt

**Why This Matters**:
- Without date context, the agent might use its training cutoff date
- Critical for queries about "today", "current", "latest", "now"
- Examples: weather, news, stock prices, events

**Best Practice**: Always provide temporal context for production systems!

## Step 5: Create the Agent Using LangChain v1 API

### NEW: One-Line Agent Creation!

The new `create_agent()` function replaces the old multi-step process:

**Old Way (Deprecated):**
```python
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
```

**New Way (LangChain v1):**
```python
agent = create_agent(model="gpt-4o", tools=tools, system_prompt=prompt)
```

### Benefits:
✅ Simpler API - one function instead of two
✅ Built on LangGraph - automatic persistence and streaming
✅ Better error handling
✅ Human-in-the-loop capabilities built-in
✅ Easier customization through middleware

### Example: See the Difference

**Before (without date):**
```
Agent searches: "London weather October 20, 2023"  ❌ Wrong date!
```

**After (with date in prompt):**
```
Agent searches: "London weather December 12, 2025"  ✅ Correct date!
```

The agent now automatically includes the correct current date in time-sensitive queries!

In [27]:
# Create the agent using the new LangChain v1 API
agent = create_agent(
    model=llm_with_tools,              # Can also use string like "gpt-4o"
    tools=tools,            # List of tools
    system_prompt=system_prompt,  # Simple string prompt
)

print("✅ Agentic RAG system created with LangChain v1 API!")
print("\n🤖 Agent capabilities:")
print("   - Reason about tool selection")
print("   - Search local knowledge base")
print("   - Search the web for current information")
print("   - Combine multiple sources")
print("   - Built on LangGraph for reliability")
print("   - Automatic streaming and persistence")
print("\n📝 Note: The agent returns a graph that can be invoked with messages")

✅ Agentic RAG system created with LangChain v1 API!

🤖 Agent capabilities:
   - Reason about tool selection
   - Search local knowledge base
   - Search the web for current information
   - Combine multiple sources
   - Built on LangGraph for reliability
   - Automatic streaming and persistence

📝 Note: The agent returns a graph that can be invoked with messages


## Step 6: Test the Agent - Example Queries

Let's test different scenarios to see how the agent reasons and decides which tool to use.

### Understanding the Agent Reasoning Loop:

The agent follows the **ReAct (Reasoning + Acting) pattern**:

```
1. 💭 THOUGHT: Agent analyzes the query and decides which tool to use
2. 🔧 ACTION: Agent calls the selected tool with appropriate input
3. 👁️  OBSERVATION: Agent receives and processes the tool's output
4. 🔄 REPEAT: If needed, agent can use more tools
5. ✅ ANSWER: Agent synthesizes all information into final response
```

We'll visualize this loop for each query below!

In [28]:
# Helper function to visualize agent reasoning loop
def visualize_agent_reasoning(query: str):
    """
    Visualizes the complete reasoning loop of the agent showing:
    - Which tools are selected
    - What input is passed to each tool
    - What observations the agent receives
    - How the agent synthesizes the final answer
    """
    print(f"\n{'='*80}")
    print(f"🔍 QUERY: {query}")
    print(f"{'='*80}\n")
    print("🤖 AGENT REASONING LOOP (ReAct Pattern):\n")
    
    step_count = 0
    tool_calls_count = 0
    
    for event in agent.stream(
        {"messages": [HumanMessage(content=query)]},
        stream_mode="values"
    ):
        messages = event.get("messages", [])
        if messages:
            last_msg = messages[-1]
            
            # Agent decides to use a tool (THOUGHT + ACTION)
            if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                step_count += 1
                print(f"{'━'*80}")
                print(f"📍 STEP {step_count}: THOUGHT + ACTION")
                print(f"{'━'*80}")
                print(f"💭 AGENT THINKING: Let me analyze this query and decide which tool to use...\n")
                
                for tool_call in last_msg.tool_calls:
                    tool_calls_count += 1
                    print(f"🎯 DECISION: I will use '{tool_call['name']}' tool")
                    print(f"📋 REASON: ", end="")
                    
                    # Provide reasoning based on tool selected
                    if "knowledge_base" in tool_call['name']:
                        print("This query is about concepts in my knowledge base")
                    elif "tavily" in tool_call['name']:
                        print("This requires current/real-time information from the web")
                    else:
                        print("This tool will help me gather the needed information")
                    
                    print(f"\n🔧 TOOL: {tool_call['name']}")
                    print(f"📥 INPUT: {tool_call['args']}")
                    print()
            
            # Agent receives tool output (OBSERVATION)
            elif type(last_msg).__name__ == 'ToolMessage':
                print(f"{'━'*80}")
                print(f"📍 STEP {step_count}: OBSERVATION")
                print(f"{'━'*80}")
                print(f"👁️  AGENT OBSERVING: Here's what the tool returned...\n")
                
                # Show truncated output
                output = str(last_msg.content)
                if len(output) > 400:
                    print(f"📤 TOOL OUTPUT (truncated):\n{output[:400]}...\n")
                else:
                    print(f"📤 TOOL OUTPUT:\n{output}\n")
                
                print(f"💭 AGENT THINKING: Let me process this information...")
                print(f"   → Do I have enough information to answer?")
                print(f"   → Or do I need to use another tool?\n")
    
    # Get final result
    result = agent.invoke({"messages": [HumanMessage(content=query)]})
    
    print(f"{'━'*80}")
    print(f"📍 FINAL STEP: SYNTHESIS")
    print(f"{'━'*80}")
    print(f"✅ AGENT CONCLUSION: I have gathered sufficient information!\n")
    print(f"📊 SUMMARY:")
    print(f"   • Total tools used: {tool_calls_count}")
    print(f"   • Reasoning steps: {step_count}")
    print(f"\n{'='*80}")
    print(f"💬 FINAL ANSWER:")
    print(f"{'='*80}")
    print(result['messages'][-1].content)
    print(f"\n{'='*80}\n")
    
    return result

print("✅ Helper function created! Use visualize_agent_reasoning(query) to see the complete reasoning loop.")

✅ Helper function created! Use visualize_agent_reasoning(query) to see the complete reasoning loop.


### Test 1: Question Requiring Local Knowledge Base

**Expected Behavior:** Agent should use the `knowledge_base_search` tool since this information is in our vector database.

**New Invocation Pattern:** The agent expects messages in a dict format with a "messages" key.

In [29]:
from langchain_core.messages import HumanMessage

query1 = "What is LangChain and what are its main components?"

# Use the helper function to visualize reasoning
result1 = visualize_agent_reasoning(query1)


🔍 QUERY: What is LangChain and what are its main components?

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'knowledge_base_search' tool
📋 REASON: This query is about concepts in my knowledge base

🔧 TOOL: knowledge_base_search
📥 INPUT: {'query': 'LangChain main components'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the tool returned...

📤 TOOL OUTPUT (truncated):
Document 1:
LangChain is a framework for developing applications powered by language models. 
        It enables applications that are context-aware and can reason. LangCha

### Test 2: Question Requiring Web Search

**Expected Behavior:** Agent should use the `tavily_search_results_json` tool since this requires current information.

In [30]:
query2 = "What are the latest updates in GPT-4 in 2025?"

# Use the helper function to visualize reasoning
result2 = visualize_agent_reasoning(query2)


🔍 QUERY: What are the latest updates in GPT-4 in 2025?

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'tavily_search_results_json' tool
📋 REASON: This requires current/real-time information from the web

🔧 TOOL: tavily_search_results_json
📥 INPUT: {'query': 'latest updates in GPT-4 2025'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the tool returned...

📤 TOOL OUTPUT (truncated):
[{"title": "ChatGPT — Release Notes: 2025-March-27 - GPT-4o a new update", "url": "https://community.openai.com/t/chatgpt-release-notes-2025-march-27-gpt-4o-a

In [18]:
query2 = "Tell me something about google antigravity?"

# Use the helper function to visualize reasoning
result2 = visualize_agent_reasoning(query2)


🔍 QUERY: Tell me something about google antigravity?

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'tavily_search_results_json' tool
📋 REASON: This requires current/real-time information from the web

🔧 TOOL: tavily_search_results_json
📥 INPUT: {'query': 'Google antigravity December 15, 2025'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the tool returned...

📤 TOOL OUTPUT (truncated):
[{"title": "Google Antigravity - Wikipedia", "url": "https://en.wikipedia.org/wiki/Google_Antigravity", "content": "Google Antigravity is an AI-powered 

### Test 3: Question Potentially Requiring Both Tools

**Expected Behavior:** Agent might use both tools to combine local knowledge with current information.

In [19]:
query3 = "How does RAG work and what are some recent improvements to RAG systems?"

# This query might trigger MULTIPLE tool calls - watch the reasoning loop!
result3 = visualize_agent_reasoning(query3)


🔍 QUERY: How does RAG work and what are some recent improvements to RAG systems?

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'knowledge_base_search' tool
📋 REASON: This query is about concepts in my knowledge base

🔧 TOOL: knowledge_base_search
📥 INPUT: {'query': 'How does Retrieval-Augmented Generation (RAG) work?'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the tool returned...

📤 TOOL OUTPUT (truncated):
Document 1:
Retrieval-Augmented Generation (RAG) is a technique that combines the power of 
        large language models with

In [31]:
query3 = "What is 10-5?"

# This query might trigger MULTIPLE tool calls - watch the reasoning loop!
result3 = visualize_agent_reasoning(query3)


🔍 QUERY: What is 10-5?

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'subtract' tool
📋 REASON: This tool will help me gather the needed information

🔧 TOOL: subtract
📥 INPUT: {'a': 10, 'b': 5}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the tool returned...

📤 TOOL OUTPUT:
5

💭 AGENT THINKING: Let me process this information...
   → Do I have enough information to answer?
   → Or do I need to use another tool?

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 FINAL STEP: SYNTHESIS
━━━━━━━━━━━━━━━━━━━━

In [33]:
query3 = "What is 30 multiply by 30 minus 200?What are the recent research paper in mathematics"

# This query might trigger MULTIPLE tool calls - watch the reasoning loop!
result3 = visualize_agent_reasoning(query3)


🔍 QUERY: What is 30 multiply by 30 minus 200?What are the recent research paper in mathematics

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'multiply' tool
📋 REASON: This tool will help me gather the needed information

🔧 TOOL: multiply
📥 INPUT: {'a': 30, 'b': 30}

🎯 DECISION: I will use 'tavily_search_results_json' tool
📋 REASON: This requires current/real-time information from the web

🔧 TOOL: tavily_search_results_json
📥 INPUT: {'query': 'recent research papers in mathematics 2025'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the 

In [25]:
query3 = "How is the weather in Delhi today?"

# This query might trigger MULTIPLE tool calls - watch the reasoning loop!
result3 = visualize_agent_reasoning(query3)


🔍 QUERY: How is the weather in Delhi today?

🤖 AGENT REASONING LOOP (ReAct Pattern):

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: THOUGHT + ACTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💭 AGENT THINKING: Let me analyze this query and decide which tool to use...

🎯 DECISION: I will use 'tavily_search_results_json' tool
📋 REASON: This requires current/real-time information from the web

🔧 TOOL: tavily_search_results_json
📥 INPUT: {'query': 'Delhi weather December 16 2025'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📍 STEP 1: OBSERVATION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
👁️  AGENT OBSERVING: Here's what the tool returned...

📤 TOOL OUTPUT (truncated):
[{"title": "Delhi weather in December 2025 - Weather25.com", "url": "https://www.weather25.com/asia/india/delhi?page=month&month=December", "content": "| 14 Image 41: 

### Test 4: Complex Reasoning Question

**Expected Behavior:** Agent should demonstrate multi-step reasoning.

In [ ]:
query4 = "Explain how LangChain agents work and compare them to traditional RAG approaches"

# Complex query requiring deep reasoning
result4 = visualize_agent_reasoning(query4)

## Step 7: Analyze Agent Reasoning and Messages

Let's examine the message flow to understand how the agent reasoned through the queries.

In LangChain v1, the agent returns all messages including:
- **HumanMessage**: User input
- **AIMessage**: Agent's reasoning and tool calls
- **ToolMessage**: Results from tool execution

### Test 5: Multi-Tool Reasoning Example

This query will likely require the agent to use **BOTH tools** - demonstrating advanced multi-step reasoning!

In [ ]:
query5 = "What is RAG and how is it being used in AI applications today?"

print("⚡ SPECIAL CASE: This query requires BOTH local knowledge AND current web information!")
print("Watch how the agent uses multiple tools in sequence!\n")

# This should demonstrate multi-tool usage
result5 = visualize_agent_reasoning(query5)

In [ ]:
# Function to display agent reasoning steps
def analyze_agent_reasoning(result, query_name):
    print(f"\n{'='*80}")
    print(f"🧠 REASONING ANALYSIS: {query_name}")
    print(f"{'='*80}\n")
    
    messages = result['messages']
    print(f"📊 Total messages: {len(messages)}\n")
    
    for i, msg in enumerate(messages, 1):
        print(f"\n--- Message {i}: {type(msg).__name__} ---")
        
        if hasattr(msg, 'content') and msg.content:
            print(f"💬 Content: {msg.content[:200]}...")
        
        # Check for tool calls in AIMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tool_call in msg.tool_calls:
                print(f"🔧 Tool Called: {tool_call['name']}")
                print(f"📥 Tool Input: {tool_call['args']}")
        
        # Check for tool results
        if type(msg).__name__ == 'ToolMessage':
            print(f"📤 Tool Output: {str(msg.content)[:200]}...")
        
        print("-" * 80)

# Analyze the reasoning for query 1
analyze_agent_reasoning(result1, "Query 1 - Local Knowledge")

---

## Understanding the Agent Reasoning Loop

### What You Just Saw: The ReAct Pattern

The agent follows a **ReAct (Reasoning + Acting)** loop for each query:

```
┌─────────────────────────────────────────────────────────────┐
│                    USER QUERY RECEIVED                       │
└─────────────────┬───────────────────────────────────────────┘
                  │
                  ▼
          ┌───────────────┐
          │  💭 THOUGHT   │  Agent analyzes the query
          │  (Reasoning)  │  "What information do I need?"
          └───────┬───────┘  "Which tool should I use?"
                  │
                  ▼
          ┌───────────────┐
          │  🔧 ACTION    │  Agent selects and calls a tool
          │  (Tool Call)  │  with appropriate input
          └───────┬───────┘
                  │
                  ▼
          ┌───────────────┐
          │ 👁️ OBSERVATION│  Agent receives tool output
          │  (Tool Result)│  and processes the information
          └───────┬───────┘
                  │
                  ▼
          ┌───────────────┐
          │   🤔 EVALUATE │  "Do I have enough info?"
          └───────┬───────┘
                  │
         ┌────────┴────────┐
         │                 │
    Need more info?   Have enough?
         │                 │
         ▼                 ▼
    ┌─────────┐      ┌──────────┐
    │ REPEAT  │      │ ✅ ANSWER│
    │  LOOP   │      │ (Synthesis)│
    └─────────┘      └──────────┘
```

### Key Observations from Tests:

1. **Tool Selection Intelligence**: 
   - Agent chose `knowledge_base_search` for conceptual questions
   - Agent chose `tavily_search_results_json` for current events

2. **Multi-Step Reasoning**: 
   - Some queries required multiple tool calls
   - Agent evaluated each result before deciding next step

3. **Context Synthesis**: 
   - Agent combined information from multiple sources
   - Final answer was coherent and comprehensive

4. **Transparency**:
   - Every decision and tool call is visible
   - You can see exactly how the agent "thinks"

---

## Step 7: Deep Dive - Analyzing Message Flow

Let's examine the complete message history to understand the agent's internal communication.

In [20]:
# Detailed message analysis function
def analyze_message_flow(result, query_name):
    """
    Analyzes the complete message flow showing all message types:
    - HumanMessage: User input
    - AIMessage: Agent's decisions and reasoning
    - ToolMessage: Tool execution results
    """
    print(f"\n{'='*80}")
    print(f"🔬 MESSAGE FLOW ANALYSIS: {query_name}")
    print(f"{'='*80}\n")
    
    messages = result['messages']
    print(f"📊 Total messages in conversation: {len(messages)}\n")
    
    for i, msg in enumerate(messages, 1):
        msg_type = type(msg).__name__
        print(f"\n{'─'*80}")
        print(f"Message #{i}: {msg_type}")
        print(f"{'─'*80}")
        
        if msg_type == 'HumanMessage':
            print(f"👤 USER INPUT:")
            print(f"   {msg.content}\n")
        
        elif msg_type == 'AIMessage':
            # Check if this is a tool-calling message or final response
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                print(f"🤖 AGENT DECISION: Using tools")
                for j, tool_call in enumerate(msg.tool_calls, 1):
                    print(f"\n   Tool Call #{j}:")
                    print(f"   └─ Tool: {tool_call['name']}")
                    print(f"   └─ Arguments: {tool_call['args']}")
            else:
                print(f"🤖 AGENT FINAL RESPONSE:")
                content = msg.content[:200] if len(msg.content) > 200 else msg.content
                print(f"   {content}...")
        
        elif msg_type == 'ToolMessage':
            print(f"🔧 TOOL RESULT:")
            content = str(msg.content)[:200] if len(str(msg.content)) > 200 else str(msg.content)
            print(f"   {content}...")
            if hasattr(msg, 'name'):
                print(f"   └─ Tool used: {msg.name}")
    
    print(f"\n{'='*80}\n")

# Analyze one of our previous results
analyze_message_flow(result1, "Query 1 - LangChain Components")


🔬 MESSAGE FLOW ANALYSIS: Query 1 - LangChain Components

📊 Total messages in conversation: 4


────────────────────────────────────────────────────────────────────────────────
Message #1: HumanMessage
────────────────────────────────────────────────────────────────────────────────
👤 USER INPUT:
   What is LangChain and what are its main components?


────────────────────────────────────────────────────────────────────────────────
Message #2: AIMessage
────────────────────────────────────────────────────────────────────────────────
🤖 AGENT DECISION: Using tools

   Tool Call #1:
   └─ Tool: knowledge_base_search
   └─ Arguments: {'query': 'LangChain main components'}

────────────────────────────────────────────────────────────────────────────────
Message #3: ToolMessage
────────────────────────────────────────────────────────────────────────────────
🔧 TOOL RESULT:
   Document 1:
LangChain is a framework for developing applications powered by language models. 
        It enables applic

### Key Insights from Message Flow Analysis:

1. **Message Types**:
   - `HumanMessage`: Contains user query
   - `AIMessage` (with tool_calls): Agent's decision to use tools
   - `ToolMessage`: Results from tool execution
   - `AIMessage` (without tool_calls): Final synthesized answer

2. **Conversation Structure**:
   ```
   HumanMessage → AIMessage (tool decision) → ToolMessage → AIMessage (final answer)
   ```

3. **Why This Matters**:
   - **Transparency**: See every decision the agent makes
   - **Debugging**: Identify where issues occur in the reasoning chain
   - **Optimization**: Understand which tools are used most frequently
   - **Auditing**: Track all tool calls for compliance/monitoring

---

---

## Bonus: Streaming Agent Responses in Real-Time

One of the key benefits of LangChain v1 built on LangGraph is **native streaming support**.

Watch the agent think and act in real-time!

# Streaming example with real-time reasoning visualization
import time

query_stream = "What is vector database and why is it important for RAG?"

print("🌊 STREAMING MODE: Watch the agent think in REAL-TIME!\n")
print(f"{'='*80}")
print(f"🔍 QUERY: {query_stream}")
print(f"{'='*80}\n")

chunk_count = 0
for chunk in agent.stream(
    {"messages": [HumanMessage(content=query_stream)]},
    stream_mode="values"  # Stream complete state updates
):
    chunk_count += 1
    messages = chunk.get("messages", [])
    
    if messages:
        last_msg = messages[-1]
        msg_type = type(last_msg).__name__
        
        # Show different visualizations based on message type
        if msg_type == "AIMessage" and hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
            print(f"⚡ Chunk #{chunk_count}: Agent selecting tool...")
            for tool_call in last_msg.tool_calls:
                print(f"   🔧 {tool_call['name']}")
        
        elif msg_type == "ToolMessage":
            print(f"⚡ Chunk #{chunk_count}: Tool execution complete")
            print(f"   📤 Result received ({len(str(last_msg.content))} chars)")
        
        elif msg_type == "AIMessage" and last_msg.content:
            print(f"⚡ Chunk #{chunk_count}: Generating final answer...")
    
    print(f"{'.'*40}")

print(f"\n✅ Streaming complete! Total chunks: {chunk_count}")
print(f"\n{'='*80}")

### Streaming Benefits:

✅ **Real-time feedback**: Users see progress immediately
✅ **Better UX**: No waiting for complete response
✅ **Debugging**: Watch the agent's reasoning unfold
✅ **Monitoring**: Track agent behavior live

### Stream Modes Available:

- `"values"`: Complete state at each step
- `"updates"`: Only what changed since last step  
- `"messages"`: Individual messages as they're created

---

In [ ]:
# Example setup using NEW create_agent API
print("""
# NEW LangChain v1 approach with @tool decorator
from langchain.tools import tool
from langchain_experimental.tools import PythonREPLTool

@tool
def calculator(expression: str) -> str:
    '''Evaluate a mathematical expression and return the result.'''
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

python_repl = PythonREPLTool()

# Create agent with NEW API
agent_with_calc = create_agent(
    model="gpt-4o",
    tools=[knowledge_base_search, calculator, python_repl],
    system_prompt="You are a technical assistant that can search docs and perform calculations."
)

# Agent workflow:
# 1. Retrieve information about GPT-3 training
# 2. Use calculator for simple math (355 GPU-years * 365 days * 24 hours * $2)
# 3. Use Python REPL for complex calculations
""")

### Example 3: SQL Database + Retriever + Web Search

**Use Case:** Answer questions about structured data with contextual information

**Tools:**
- SQL Database Tool: Query structured data
- Retriever: Access unstructured documentation
- Web Search: Get latest information

**Scenario:** "What were our top-selling products last quarter and what do experts say about market trends?"

In [ ]:
# Example setup with NEW API
print("""
# Using the @tool decorator for custom retrieval tools
from langchain.tools import tool

# Create multiple vector stores
docs_vectorstore = Chroma(..., collection_name="technical_docs")
support_vectorstore = Chroma(..., collection_name="customer_support")
code_vectorstore = Chroma(..., collection_name="code_examples")

# Create custom tool functions with @tool decorator
@tool
def search_technical_docs(query: str) -> str:
    '''Search official technical documentation and API references.'''
    docs = docs_vectorstore.as_retriever().invoke(query)
    return "\\n\\n".join([doc.page_content for doc in docs])

@tool
def search_support_tickets(query: str) -> str:
    '''Search past customer support tickets and solutions.'''
    docs = support_vectorstore.as_retriever().invoke(query)
    return "\\n\\n".join([doc.page_content for doc in docs])

@tool
def search_code_examples(query: str) -> str:
    '''Search code examples and implementation patterns.'''
    docs = code_vectorstore.as_retriever().invoke(query)
    return "\\n\\n".join([doc.page_content for doc in docs])

# Create agent with NEW API - much simpler!
multi_db_agent = create_agent(
    model="gpt-4o",
    tools=[search_technical_docs, search_support_tickets, search_code_examples, web_search_tool],
    system_prompt="You are a helpful assistant that can search multiple knowledge bases."
)
""")

### Example 4: API Integration + RAG

**Use Case:** Real-time data from external APIs combined with knowledge base

**Tools:**
- Weather API: Current weather data
- Stock API: Live stock prices
- News API: Latest news
- Retriever: Historical context and analysis

**Scenario:** "How is the current weather affecting airline stock prices based on historical patterns?"

In [ ]:
# Example setup with NEW API
print("""
from langchain_community.utilities import SQLDatabase
from langchain.tools import tool
import requests

# Connect to database
db = SQLDatabase.from_uri("sqlite:///sales.db")

# Create custom tools with @tool decorator
@tool
def query_sales_database(sql_query: str) -> str:
    '''Execute SQL queries on the sales database for product information, revenue, and customer data.'''
    try:
        result = db.run(sql_query)
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}"

# Create agent with NEW API
sql_rag_agent = create_agent(
    model="gpt-4o",
    tools=[query_sales_database, knowledge_base_search, web_search_tool],
    system_prompt='''You are a business analyst assistant. 
    Use the SQL database for quantitative data, knowledge base for product context, 
    and web search for market trends.'''
)

# Agent workflow:
# 1. Query SQL database for sales data
# 2. Retrieve product documentation for context
# 3. Search web for market trends
# 4. Synthesize comprehensive answer
""")

### Example 5: Document Analysis + Web Research

**Use Case:** Analyze uploaded documents with external context

**Tools:**
- PDF Retriever: Search through uploaded PDFs
- CSV Analyzer: Analyze spreadsheet data
- Web Search: Find related information
- Wikipedia: Get background knowledge

**Scenario:** "Analyze this research paper and find related recent studies"

In [ ]:
# Example setup with NEW API
print("""
from langchain.tools import tool
import requests

# Create custom API tools with @tool decorator
@tool
def get_weather(location: str) -> str:
    '''Get current weather conditions for any location.'''
    try:
        response = requests.get(f"https://api.weather.com/{location}", timeout=10)
        response.raise_for_status()
        return response.text
    except Exception as e:
        return f"Error fetching weather: {str(e)}"

@tool
def get_stock_price(symbol: str) -> str:
    '''Get real-time stock prices and trading data for a given stock symbol.'''
    try:
        response = requests.get(f"https://api.stocks.com/{symbol}", timeout=10)
        response.raise_for_status()
        return response.text
    except Exception as e:
        return f"Error fetching stock data: {str(e)}"

# Create agent with NEW API
api_rag_agent = create_agent(
    model="gpt-4o",
    tools=[get_weather, get_stock_price, knowledge_base_search, web_search_tool],
    system_prompt="You combine real-time API data with historical knowledge to provide insights."
)

# Example usage:
# result = api_rag_agent.invoke({
#     "messages": [HumanMessage(content="How is weather affecting airline stocks today?")]
# })
""")

### Example 6: Hybrid Search (Dense + Sparse)

**Use Case:** Combine semantic search with keyword search

**Tools:**
- Dense Retriever: Semantic similarity search (embeddings)
- Sparse Retriever: Keyword/BM25 search
- Reranker: Rerank results for better relevance

**Benefits:**
- Better recall (find more relevant documents)
- Handle exact matches and semantic matches
- Improved accuracy

In [ ]:
# Example setup with NEW API
print("""
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import tool

# Load and process PDF
pdf_loader = PyPDFLoader("research_paper.pdf")
pdf_docs = pdf_loader.load_and_split()
pdf_vectorstore = Chroma.from_documents(pdf_docs, embeddings)
pdf_retriever = pdf_vectorstore.as_retriever()

# Create custom PDF search tool
@tool
def search_pdf_document(query: str) -> str:
    '''Search through the uploaded PDF research paper.'''
    docs = pdf_retriever.invoke(query)
    return "\\n\\n".join([doc.page_content for doc in docs])

# Wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Create agent with NEW API
doc_analysis_agent = create_agent(
    model="gpt-4o",
    tools=[search_pdf_document, wikipedia, web_search_tool],
    system_prompt="Analyze documents and find related research from multiple sources."
)
""")

### Example 7: Memory-Enhanced Agentic RAG

**Use Case:** Multi-turn conversations with context awareness

**Components:**
- Conversation Memory: Remember previous interactions
- Retriever: Access knowledge base
- Web Search: Current information

**Benefits:**
- Follow-up questions work naturally
- Reference previous answers
- Build on previous context

In [ ]:
# Example setup with NEW API
print("""
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.tools import tool

# Dense retriever (semantic)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Sparse retriever (keyword)
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 5

# Ensemble retriever (combines both)
ensemble_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, bm25_retriever],
    weights=[0.5, 0.5]  # Equal weight to both methods
)

# Create hybrid search tool
@tool
def hybrid_search(query: str) -> str:
    '''Search using both semantic similarity and keyword matching for better accuracy.'''
    docs = ensemble_retriever.invoke(query)
    return "\\n\\n".join([doc.page_content for doc in docs])

# Create agent with NEW API
hybrid_agent = create_agent(
    model="gpt-4o",
    tools=[hybrid_search, web_search_tool],
    system_prompt="Use hybrid search for comprehensive retrieval combining semantic and keyword matching."
)
""")

### Example 7: Conversational Agentic RAG with LangGraph Persistence

**Use Case:** Multi-turn conversations with context awareness

**NEW in LangChain v1:** Built-in persistence through LangGraph checkpointing!

**Benefits:**
- Follow-up questions work naturally
- Reference previous answers
- No need for separate memory management
- Automatic conversation state handling

---

## What Changed in LangChain v1? Migration Guide

### Old API (Deprecated) → New API (Current)

| Old (create_react_agent/create_tool_calling_agent) | New (create_agent) |
|-----------------------------------------------------|---------------------|
| Multiple imports and setup | Single import: `from langchain.agents import create_agent` |
| Separate prompt template | Simple `system_prompt` string |
| Create agent + Create executor (2 steps) | Single `create_agent()` call |
| Manual memory management | Built-in persistence via LangGraph |
| Complex streaming setup | Native `.stream()` support |
| `agent_executor.invoke({"input": query})` | `agent.invoke({"messages": [HumanMessage(content=query)]})` |

### Key Benefits of v1:

✅ **Simpler API**: Less boilerplate code
✅ **Built on LangGraph**: Automatic persistence, streaming, human-in-the-loop
✅ **Better reliability**: Improved error handling and retries
✅ **Tool decorator**: Clean `@tool` syntax for custom tools
✅ **Message-based**: Consistent message interface
✅ **Middleware support**: Easy customization and extension

---

In [ ]:
# Example: Conversational agent with automatic persistence
print("""
# NEW LangChain v1: Memory is built-in through LangGraph!
from langgraph.checkpoint.memory import MemorySaver

# Create checkpointer for conversation history
memory = MemorySaver()

# Create agent with memory support
conversational_agent = create_agent(
    model="gpt-4o",
    tools=[knowledge_base_search, web_search_tool],
    system_prompt="You are a helpful assistant with memory of our conversation.",
    checkpointer=memory  # Add persistence!
)

# Multi-turn conversation with thread_id
thread_id = "conversation-123"

# First question
response1 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="What is LangChain?")]},
    config={"configurable": {"thread_id": thread_id}}
)

# Follow-up question - agent remembers context!
response2 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="How does it compare to other frameworks?")]},
    config={"configurable": {"thread_id": thread_id}}
)

# Agent automatically maintains conversation history!
# No need for manual memory management like before
""")

---

## Key Takeaways and Best Practices

### 1. Tool Design (Updated for v1)
- **Use @tool decorator**: Cleaner and more maintainable
- **Clear docstrings**: Help agent understand when to use each tool
- **Type hints**: Required for proper tool argument parsing
- **Good naming**: Use descriptive, intuitive function names

### 2. Prompt Engineering
- **Use system_prompt**: Simple string instead of complex templates
- **Tool guidelines**: Provide clear criteria for tool selection
- **Be explicit**: Tell agent how to reason and respond

### 3. Message Handling (NEW)
- **Message format**: Use `{"messages": [HumanMessage(content="...")]}` 
- **Access results**: Final answer in `result['messages'][-1].content`
- **Conversation threads**: Use thread_id for multi-turn conversations

### 4. Performance Optimization
- **Chunk size**: Balance between context and precision
- **Number of retrievals**: More isn't always better (k=3 is often good)
- **Caching**: Cache embeddings and frequent queries
- **Streaming**: Use `.stream()` for better user experience

### 5. LangGraph Features (Built-in with v1)
- **Persistence**: Add `checkpointer` for conversation memory
- **Streaming**: Multiple modes ("values", "updates", "messages")
- **Human-in-the-loop**: Pause for approval before actions
- **Time travel**: Rewind and explore alternate paths

### 6. Production Considerations
- **Error handling**: Built-in retry and error handling
- **Rate limiting**: Prevent API abuse
- **Logging**: Use LangSmith for tracing and debugging
- **Security**: Sanitize tool inputs and outputs
- **Monitoring**: Track costs, latency, and success rates

---

## Conclusion

Agentic RAG with LangChain v1 represents a powerful evolution in building intelligent applications:

✅ **Simpler API**: `create_agent()` replaces complex multi-step setup

✅ **Dynamic Tool Selection**: Agent intelligently chooses the right tool for each query

✅ **Multi-Source Information**: Combines vector DB, web search, APIs, and more

✅ **Built on LangGraph**: Automatic persistence, streaming, and reliability

✅ **Better Developer Experience**: Less boilerplate, clearer patterns

✅ **Production-Ready**: Built-in error handling, retries, and monitoring support

### Next Steps:
1. Experiment with different tool combinations for your use case
2. Add custom tools using the `@tool` decorator
3. Implement conversation persistence with checkpointers
4. Use LangSmith for tracing and evaluation
5. Deploy with proper monitoring and rate limiting

### Resources:
- **LangChain Docs**: https://docs.langchain.com
- **LangGraph Docs**: https://langchain-ai.github.io/langgraph/
- **LangSmith**: https://smith.langchain.com (for tracing and evaluation)
- **Community**: https://github.com/langchain-ai/langchain

---

**Happy Building with Agentic RAG! 🚀**